# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import pandas_datareader.data as pdr
import yfinance as yf
import matplotlib
from scipy import stats
yf.pdr_override()

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [24]:
stocks =pd.read_csv("constituents.csv")
for i in range(0,len(stocks["Symbol"])):
    if "." in stocks["Symbol"][i]:
        stocks["Symbol"][i]=stocks["Symbol"][i].replace(".","-")
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary
499,ZBRA,Zebra Technologies,Information Technology
500,ZBH,Zimmer Biomet,Health Care
501,ZION,Zions Bancorporation,Financials


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol="AAPL"
data=pdr.get_quote_yahoo(symbol)["twoHundredDayAverageChange"]
price=data["price"].to_dict()[symbol]
marketcap=data["marketCap"].to_dict()[symbol]
price,marketcap


(134.76, 2143775490048)

In [8]:
data.columns

Index(['language', 'region', 'quoteType', 'typeDisp', 'quoteSourceName',
       'triggerable', 'customPriceAlertConfidence', 'marketState', 'exchange',
       'shortName', 'longName', 'messageBoardId', 'exchangeTimezoneName',
       'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'market',
       'esgPopulated', 'currency', 'regularMarketChangePercent',
       'regularMarketPrice', 'firstTradeDateMilliseconds', 'priceHint',
       'postMarketChangePercent', 'postMarketTime', 'postMarketPrice',
       'postMarketChange', 'regularMarketChange', 'regularMarketTime',
       'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow',
       'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask',
       'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency',
       'regularMarketOpen', 'averageDailyVolume3Month',
       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',
       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',
       'fiftyTwoWeekHighCh

In [22]:
data=pdr.get_quote_yahoo(symbol)
data["twoHundredDayAverageChangePercent"],data["twoHundredDayAverageChange"],data["twoHundredDayAverage"]-data["price"]

(AAPL   -0.100354
 Name: twoHundredDayAverageChangePercent, dtype: float64,
 AAPL   -15.032257
 Name: twoHundredDayAverageChange, dtype: float64,
 AAPL    15.03225
 dtype: float64)

In [19]:
data['trailingAnnualDividendRate']

AAPL    0.9
Name: trailingAnnualDividendRate, dtype: float64

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [25]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [43]:
final_df=pd.DataFrame(columns=my_columns)
for symbol_group in symbol_groups:
    data=pdr.get_quote_yahoo(symbol_group)[["price","twoHundredDayAverageChangePercent"]]
    data=data.reset_index()
    data["nan"]=np.nan
    data=data.set_axis(my_columns,axis=1)
    final_df=pd.concat([final_df,data])

In [ ]:
final_df=final_df.reset_index()
final_df=final_df.drop("index",axis=1)
final_df

In [53]:
final_df=final_df.drop(["index","level_0"],axis=1)

In [54]:
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,129.51,-0.021934,NaN
1,AOS,62.13,0.071633,NaN
2,ABT,113.51,0.049441,NaN
3,ABBV,153.60,0.021140,NaN
4,ACN,282.14,-0.021954,NaN
...,...,...,...,...
498,YUM,130.22,0.100335,NaN
499,ZBRA,293.70,-0.037636,NaN
500,ZBH,123.38,0.069312,NaN
501,ZION,51.39,-0.045135,NaN


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [58]:
final_df.sort_values("One-Year Price Return",ascending =False,inplace=True)
final_df=final_df[:50]
final_df.reset_index(inplace=True)

In [60]:
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,195,FSLR,180.19,0.621243,NaN
1,495,WYNN,100.25,0.462967,NaN
2,279,LVS,54.97,0.417880,NaN
3,71,BA,214.13,0.358618,NaN
4,405,SLB,58.28,0.351933,NaN
5,327,NFLX,332.82,0.351525,NaN
6,213,GE,80.20,0.338819,NaN
7,421,STLD,111.52,0.313924,NaN
8,179,ETSY,134.67,0.296224,NaN
9,400,ROST,120.01,0.288296,NaN


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [65]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input("Enter the size of your portfolio:")

    try:
        float(portfolio_size)
    except ValueError:
        print ("That is not a number! \nPlease try again:")
        portfolio_size=input("Enter the size of your portfolio:")
portfolio_input()
portfolio_size

'10000000'

In [71]:
position_size=float(portfolio_size)/len(final_df.index)
for i in range(0,len(final_df)):
    final_df.loc[i,"Number of Shares to Buy"]=math.floor(position_size/final_df.loc[i,"Price"])
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,195,FSLR,180.19,0.621243,1109.0
1,495,WYNN,100.25,0.462967,1995.0
2,279,LVS,54.97,0.417880,3638.0
3,71,BA,214.13,0.358618,934.0
4,405,SLB,58.28,0.351933,3431.0
5,327,NFLX,332.82,0.351525,600.0
6,213,GE,80.20,0.338819,2493.0
7,421,STLD,111.52,0.313924,1793.0
8,179,ETSY,134.67,0.296224,1485.0
9,400,ROST,120.01,0.288296,1666.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [102]:
hqm_columns=[
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "HQM Score"
]
hqm_df=pd.DataFrame(columns=hqm_columns)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,HQM Score


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [87]:
data=pdr.get_quote_yahoo("AAPL")[["price","twoHundredDayAverageChangePercent",'fiftyDayAverageChangePercent']]
data=data.reset_index()
data["nan1"]=np.nan
data["nan2"]=np.nan
data.reindex(columns=["price","twoHundredDayAverageChangePercent","nan1",'fiftyDayAverageChangePercent',"nan2"])

,price,twoHundredDayAverageChangePercent,nan1,fiftyDayAverageChangePercent,nan2
0,134.76,-0.100354,NaN,-0.034128,NaN


In [168]:
hqm_df=pd.DataFrame(columns=hqm_columns)
for symbol_group in symbol_groups:
    data=pdr.get_quote_yahoo(symbol_group)[["price","twoHundredDayAverageChangePercent",'fiftyDayAverageChangePercent']]
    data.reset_index(inplace=True)
    data["nan1"]=np.nan
    data["nan2"]=np.nan
    data["nan3"]=np.nan
    data["nan"]=np.nan
    data=data.reindex(columns=["index","price","nan","twoHundredDayAverageChangePercent","nan1",'fiftyDayAverageChangePercent',"nan2","nan3"])
    data.set_axis(hqm_columns,axis=1,inplace=True)
    hqm_df=pd.concat([hqm_df,data])

/tmp/ipykernel_6999/3000922229.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  data.set_axis(hqm_columns,axis=1,inplace=True)
/tmp/ipykernel_6999/3000922229.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  data.set_axis(hqm_columns,axis=1,inplace=True)
/tmp/ipykernel_6999/3000922229.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  data.set_axis(hqm_columns,axis=1,inplace=True)
/tmp/ipykernel_6999/3000922229.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  data.set_axis(hqm_columns,axis=1,inplace=True)
/tmp/ipykernel_6999/3000

In [175]:
hqm_df.reset_index(inplace=True,drop=True)
hqm_df.drop(["level_0","index"],axis=1,inplace=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,HQM Score
0,MMM,129.51,NaN,-0.021934,NaN,0.032219,NaN,NaN
1,AOS,62.13,NaN,0.071633,NaN,0.054610,NaN,NaN
2,ABT,113.51,NaN,0.049441,NaN,0.065126,NaN,NaN
3,ABBV,153.60,NaN,0.021140,NaN,-0.030578,NaN,NaN
4,ACN,282.14,NaN,-0.021954,NaN,0.008324,NaN,NaN
...,...,...,...,...,...,...,...,...
498,YUM,130.22,NaN,0.100335,NaN,0.024988,NaN,NaN
499,ZBRA,293.70,NaN,-0.037636,NaN,0.136968,NaN,NaN
500,ZBH,123.38,NaN,0.069312,NaN,0.023350,NaN,NaN
501,ZION,51.39,NaN,-0.045135,NaN,0.031605,NaN,NaN


In [176]:
time_periods=[
    "One-Year",
    "Three-Month"
]
for row in hqm_df.index:
    for time_period in time_periods:
        change_col=f"{time_period} Price Return"
        percentile_col=f'{time_period} Return Percentile'
        hqm_df.loc[row,percentile_col]=stats.percentileofscore(hqm_df[change_col],hqm_df.loc[row,change_col])/100
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,HQM Score
0,MMM,129.51,NaN,-0.021934,0.296223,0.032219,0.502982,NaN
1,AOS,62.13,NaN,0.071633,0.632207,0.054610,0.673956,NaN
2,ABT,113.51,NaN,0.049441,0.554672,0.065126,0.739563,NaN
3,ABBV,153.60,NaN,0.021140,0.455268,-0.030578,0.105368,NaN
4,ACN,282.14,NaN,-0.021954,0.294235,0.008324,0.288270,NaN
...,...,...,...,...,...,...,...,...
498,YUM,130.22,NaN,0.100335,0.715706,0.024988,0.421471,NaN
499,ZBRA,293.70,NaN,-0.037636,0.252485,0.136968,0.948310,NaN
500,ZBH,123.38,NaN,0.069312,0.622266,0.023350,0.399602,NaN
501,ZION,51.39,NaN,-0.045135,0.218688,0.031605,0.493042,NaN


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [177]:
from statistics import mean
for row in hqm_df.index:
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row,f'{time_period} Return Percentile'])
    hqm_df.loc[row,"HQM Score"]=mean(momentum_percentiles)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,HQM Score
0,MMM,129.51,NaN,-0.021934,0.296223,0.032219,0.502982,0.399602
1,AOS,62.13,NaN,0.071633,0.632207,0.054610,0.673956,0.653082
2,ABT,113.51,NaN,0.049441,0.554672,0.065126,0.739563,0.647117
3,ABBV,153.60,NaN,0.021140,0.455268,-0.030578,0.105368,0.280318
4,ACN,282.14,NaN,-0.021954,0.294235,0.008324,0.288270,0.291252
...,...,...,...,...,...,...,...,...
498,YUM,130.22,NaN,0.100335,0.715706,0.024988,0.421471,0.568588
499,ZBRA,293.70,NaN,-0.037636,0.252485,0.136968,0.948310,0.600398
500,ZBH,123.38,NaN,0.069312,0.622266,0.023350,0.399602,0.510934
501,ZION,51.39,NaN,-0.045135,0.218688,0.031605,0.493042,0.355865


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [129]:
# hqm_df=hqm_df.drop("index",axis=1)

In [178]:
hqm_df.sort_values("HQM Score",ascending=False,inplace=True)
hqm_df=hqm_df[:50]
hqm_df.reset_index(inplace=True,drop=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,HQM Score
0,WYNN,100.25,NaN,0.462967,0.998012,0.220305,0.998012,0.998012
1,LVS,54.97,NaN,0.417880,0.996024,0.186632,0.984095,0.990060
2,GE,80.20,NaN,0.338819,0.988072,0.200520,0.992048,0.990060
3,UAL,51.65,NaN,0.267326,0.970179,0.218201,0.996024,0.983101
4,BA,214.13,NaN,0.358618,0.994036,0.166789,0.970179,0.982107
5,TPR,43.38,NaN,0.280849,0.978131,0.177710,0.980119,0.979125
6,FCX,45.05,NaN,0.279948,0.976143,0.174449,0.978131,0.977137
7,FSLR,180.19,NaN,0.621243,1.000000,0.135511,0.946322,0.973161
8,MKTX,329.02,NaN,0.239568,0.950298,0.192919,0.988072,0.969185
9,NFLX,332.82,NaN,0.351525,0.990060,0.119455,0.924453,0.957256


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [119]:
portfolio_input()

In [179]:
position_size=float(portfolio_size)/len(hqm_df.index)
position_size
for i in hqm_df.index:
    hqm_df.loc[i,"Number of Shares to Buy"]=math.floor(position_size/hqm_df.loc[i,"Price"])
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,HQM Score
0,WYNN,100.25,1995.0,0.462967,0.998012,0.220305,0.998012,0.998012
1,LVS,54.97,3638.0,0.417880,0.996024,0.186632,0.984095,0.990060
2,GE,80.20,2493.0,0.338819,0.988072,0.200520,0.992048,0.990060
3,UAL,51.65,3872.0,0.267326,0.970179,0.218201,0.996024,0.983101
4,BA,214.13,934.0,0.358618,0.994036,0.166789,0.970179,0.982107
5,TPR,43.38,4610.0,0.280849,0.978131,0.177710,0.980119,0.979125
6,FCX,45.05,4439.0,0.279948,0.976143,0.174449,0.978131,0.977137
7,FSLR,180.19,1109.0,0.621243,1.000000,0.135511,0.946322,0.973161
8,MKTX,329.02,607.0,0.239568,0.950298,0.192919,0.988072,0.969185
9,NFLX,332.82,600.0,0.351525,0.990060,0.119455,0.924453,0.957256


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [180]:
writer = pd.ExcelWriter("momentum_strategy.xlsx",engine="xlsxwriter")
hqm_df.to_excel(writer,sheet_name="Momentum Strategy",index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [181]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [182]:
column_formats={
    "A":["Ticker",string_template],
    "B":["Price",dollar_template],
    "C":["Number of Shares to Buy",integer_template],
    "D":["One-Year Price Return",percent_template],
    "E":["One-Year Return Percentile",percent_template],
    "F":["Three-Month Price Return",percent_template],
    "G":["Three-Month Return Percentile",percent_template],
    "H":["HQM Score",percent_template]
}

for column in column_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(f"{column}:{column}",25,column_formats[column][1])
    writer.sheets["Momentum Strategy"].write(f"{column}1",column_formats[column][0],column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [183]:
writer.save()

/tmp/ipykernel_6999/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
